# Amazon Fine Food Review Analysis

The Amazon fine food review dataset was taken from kaggle source: https://www.kaggle.com/snap/amazon-fine-food-reviews.
The dataset contains reviews on fine foods over a span of 13 years (Oct 1999 - Oct 2009). There are 568,454 reviews from 256,059 users on 74,258 products with 260 users posting more than 50 reviews.

There are 10 attributes present for each data point.
Attribute Information:
1.Id - Row Id
2.ProductId - Unique identifier for the product
3.UserId - Unqiue identifier for the user
4.ProfileName - Profile name of the user
5.HelpfulnessNumerator - Number of users who found the review helpful
6.HelpfulnessDenominator - Number of users who indicated whether they found the review helpful or not
7.Score - Rating between 1 and 5
8.Time - Timestamp for the review
9.Summary - Brief summary of the review
10.Text - Text of the review

### Objective:

The objective is to identify the polarity(positive/ negative) of a given review. The attribute Score/ Rating is a promising indicator of polarity, with a score of 4 or 5 indicating a positive review and a score of 1 or 2 indicating a negative review. A score of 3 could be considered as neutral review and could be ignored in this case. 

In [10]:
# Import necessary libraries
import sqlite3
import pandas as pd
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from gensim.models import Word2Vec
import numpy as np

In [2]:
# The data is available in both .csv format and also as SQLite database. We will be using SQLite to load data and query on it
con = sqlite3.connect('database.sqlite')

# Filter the data to avoid neutral score of 3 and set the limit according to computing power
filtered_data = pd.read_sql_query(""" SELECT * FROM reviews WHERE Score != 3 LIMIT 5000""",con)

# The Score is normalized to 0 for negative review (Score is 1 or 2) and 1 for positive review (Score is 4 or 5)
def normalize(value):
    if value < 3:
        return 0
    return 1

actual_score = filtered_data['Score']
normalized_score = actual_score.map(normalize)
filtered_data['Score'] = normalized_score

print('Shape of review dataset: ',filtered_data.shape)
filtered_data.head(2)

Shape of review dataset:  (5000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


# Exploratory Data Analysis 

### Data Cleaning: Deduplication

After going through the data, it was observed that there are some duplicate entries by the same user at the same timestamp. This should be avoided to do an unbiased analysis

In [3]:
# Sorting the data
sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

# Deduplication of data
final_data = sorted_data.drop_duplicates(subset={'UserId', 'ProfileName', 'Time', 'Text'}, keep='first', inplace=False)

print('Shape of final data: ',final_data.shape)

# Number of positive and negative reviews available
final_data['Score'].value_counts()

Shape of final data:  (4986, 10)


1    4178
0     808
Name: Score, dtype: int64

# Text Preprocessing 

Before further analysis and model creation, text preprocessing should be done.
The following preprocessing will done:
1. Remove HTML tags
2. Remove punctuation and special characters
3. Check if it's english word and length of words are greater than 2
4. Convert words to lower case
5. Remove stop words
6. Snowball stemming the word


In [4]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])
preprocessed_text = []

for text in tqdm(final_data['Text'].values):
    text = re.sub(r"http\S+", "", text) #Remove URLs
    text = BeautifulSoup(text, 'lxml').get_text() #Remove HTML tags using BeautifulSoup
    text = re.sub("\S*\d\S*", "", text).strip() #Remove digits
    text = re.sub('[^A-Za-z0-9]+', ' ', text) #Remove special characters
    text = ' '.join(e.lower() for e in text.split() if e.lower() not in stopwords) #Remove stopwords and convert to lowercase
    preprocessed_text.append(text.strip())
    
#Print sample
print(preprocessed_text[4985])


100%|████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:02<00:00, 1976.49it/s]


coffee supposedly premium tastes watery thin not good maybe old not sure waste using line bottom sitting shoes trash cans rained luggage absorb smells used not drink not buy


#  Feature Engineering

Now that text is preprocessed, features will be derived from the text using multiple methods like Bag of Words, TFIDF etc.

### Bag of Words 

The bag-of-words model is a simplifying representation used in natural language processing and information retrieval (IR). In this model, a text (such as a sentence or a document) is represented as the bag (multiset) of its words, disregarding grammar and even word order but keeping multiplicity. The bag-of-words model has also been used for computer vision.

In [5]:
#Initialize Bag of Words
bag_of_words = CountVectorizer()
bag_of_words.fit(preprocessed_text)
print('Some of the feature names: ', bag_of_words.get_feature_names()[:10])

final_bag_of_words = bag_of_words.fit_transform(preprocessed_text)
print('Shape of BOW vectorizer: ',final_bag_of_words.get_shape())

Some of the feature names:  ['aa', 'aahhhs', 'aback', 'abandon', 'abates', 'abbott', 'abby', 'abdominal', 'abiding', 'ability']
Shape of BOW vectorizer:  (4986, 12974)


### Bi-grams and N-grams 

In [6]:
# CountVectorizer documentation: 
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
bigram = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
bigram.fit(preprocessed_text)
final_bigram = bigram.fit_transform(preprocessed_text)
print('Some of the feature names: ',bigram.get_feature_names()[:10])
print('Shape of Bi-grams vectorizer: ', final_bigram.get_shape())


Some of the feature names:  ['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']
Shape of Bi-grams vectorizer:  (4986, 3029)


### TF-IDF

In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general. tf–idf is one of the most popular term-weighting schemes today; 83% of text-based recommender systems in digital libraries use tf–idf.

In [7]:
tf_idf = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf.fit(preprocessed_text)
final_tf_idf = tf_idf.transform(preprocessed_text)

print("Some of the feature names: ", tf_idf.get_feature_names()[:10])
print("Shape of TF-IDF vectorizer: ",final_tf_idf.get_shape())

Some of the feature names:  ['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']
Shape of TF-IDF vectorizer:  (4986, 3029)


### Word2Vec

Word2vec is a group of related models that are used to produce word embeddings. These models are shallow, two-layer neural networks that are trained to reconstruct linguistic contexts of words. Word2vec takes as its input a large corpus of text and produces a vector space, typically of several hundred dimensions, with each unique word in the corpus being assigned a corresponding vector in the space. Word vectors are positioned in the vector space such that words that share common contexts in the corpus are located close to one another in the space.

In [8]:
list_of_sentances =[]
for sentance in preprocessed_text:
    list_of_sentances.append(sentance.split())

w2v_model = Word2Vec(list_of_sentances,min_count=5,size=50,workers=4)
print(w2v_model.wv.most_similar('great'))
print('-'*50)
print(w2v_model.wv.most_similar('worst'))

w2v_words = list(w2v_model.wv.vocab)

[('alternative', 0.9965649247169495), ('flavorful', 0.9959824681282043), ('enjoy', 0.9957005977630615), ('want', 0.9955089688301086), ('creamy', 0.9952535033226013), ('greasy', 0.9952250719070435), ('tasty', 0.9952051043510437), ('satisfying', 0.995172917842865), ('salty', 0.9951695203781128), ('crunch', 0.9951678514480591)]
--------------------------------------------------
[('dinner', 0.99945068359375), ('spices', 0.9994158744812012), ('goes', 0.9994075894355774), ('prefer', 0.9993808269500732), ('stick', 0.9993717074394226), ('finish', 0.9993299245834351), ('lover', 0.9993246793746948), ('knows', 0.9993175268173218), ('sauces', 0.9993101954460144), ('special', 0.9993067383766174)]


### Average Word2Vec 

The vector representation of each word in a sentance or review is added to form a sentance vector which is divided by the number of words in a sentance to form the Average Word2Vec

In [12]:
avg_w2v=[] #Initialize avg w2v

for sentance in tqdm(list_of_sentances):
    sent_vector = np.zeros(50) # Initialize sentance vector
    word_count = 0 # To keep track of word count in a sentance
    for word in sentance:
        if word in w2v_words:
            vector = w2v_model.wv[word]
            sent_vector += vector
            word_count += 1
    if word_count != 0:
        sent_vector /= word_count
    avg_w2v.append(sent_vector)

print('Length of average word2vec list: ',len(avg_w2v))
print('Sample of average word2vec: ',avg_w2v[0])


100%|█████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:06<00:00, 728.98it/s]


Length of average word2vec list:  4986
Sample of average word2vec:  [ 0.23649452 -0.36871606 -0.10766503 -0.62766277 -0.10249596 -0.43541913
 -0.8812992   0.55790287 -0.0676367   0.39250815  0.45492365 -0.24507494
  0.20232831 -0.15278611  0.05318909  0.29941429 -0.2712685   0.05432077
 -0.13479393 -0.20653179 -0.45284409 -0.20329681 -0.28953561 -0.41411759
  0.19637658  0.15092351  0.39558214  0.41059417 -0.19367852  0.34257021
 -0.88124236 -0.00445258  0.35823862  0.35570899  0.21839596 -0.26053498
  0.16816067  0.13024975 -0.12458942  0.18712186 -0.3412513  -0.17419735
 -0.01994447  0.33583819 -0.42087608 -0.10069617 -0.2708039   0.12966724
 -0.36857956 -0.12163807]


### TF-IDF weighted Word2Vec

The vector representation of each word is multiplied by TF-IDF value of each word which is summed up to form a sentance vector which is then divided by the summation of TF-IDF value to form TF-IDF weighted Word2Vec.

In [14]:
# A dictionary representation with word as key and it's value as idf score
idf_dictionary = dict(zip(tf_idf.get_feature_names(),list(tf_idf.idf_)))
tf_idf_features = tf_idf.get_feature_names() # List of words/ features
tfidf_weighted_w2v = []
for sentance in tqdm(list_of_sentances):
    sentance_vector = np.zeros(50) # Initialize sentance vector
    weighted_sum = 0 # Initialize weighted sum
    for word in sentance:
        if word in w2v_words and word in tf_idf_features:
            vector = w2v_model.wv[word]
            tfidf_score = idf_dictionary[word] * (sentance.count(word)/len(sentance)) # Calculated TF-IDF score
            sentance_vector += (vector * tfidf_score)
            weighted_sum += tfidf_score
    if weighted_sum != 0:
        sentance_vector /= weighted_sum
        tfidf_weighted_w2v.append(sentance_vector)
print('Length of TF-IDF weighted Word2Vec: ',len(tfidf_weighted_w2v))
print('Sample of TF-IDF weighted Word2Vec: ',tfidf_weighted_w2v[50])


100%|█████████████████████████████████████████████████████████████████████████████| 4986/4986 [00:15<00:00, 318.60it/s]


Length of TF-IDF weighted Word2Vec:  4971
Sample of TF-IDF weighted Word2Vec:  [ 0.26384643 -0.35279481 -0.15061843 -0.67486583 -0.08501832 -0.42447237
 -0.93819549  0.55721846 -0.01252233  0.49167766  0.53034585 -0.36530875
  0.21766431 -0.22547929  0.04377294  0.35530394 -0.26705126 -0.00584531
 -0.1199931  -0.16028277 -0.55552022 -0.18299383 -0.35456699 -0.42164837
  0.22822225  0.10326524  0.40918249  0.41522415 -0.20026586  0.34801321
 -1.01382213 -0.04701484  0.42545787  0.3243582   0.32483988 -0.29064406
  0.2970001   0.22330414 -0.12051194  0.20783974 -0.36212396 -0.24012246
 -0.0504149   0.42182765 -0.45309716 -0.14031337 -0.28156109  0.2036233
 -0.35588629 -0.10450471]
